In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from loader import load_ferdosi

dataset_dir = '/Users/Alireza/Desktop/Current Semester/Deep Learning/Assignments/DL_HW4/ferdosi.txt'
x_train, x_test, y_train, y_test, dictionary = load_ferdosi(dataset_dir)

In [4]:
print (len(dictionary))
print (len(x_train), len(x_test))
print (len(y_train), len(y_test))
print(x_train[0])

40
44648 4961
44648 4961
['_PAD_' '_PAD_' '_PAD_' '_PAD_' '_PAD_' '_PAD_' '_PAD_' '_PAD_' '_PAD_'
 '_PAD_' '_BOM_' 'و' 'گ' 'ر' ' ' 'د' 'ر' ' ' 'م' 'ی' 'ا' 'ن' ' ' 'د' 'و'
 ' ' 'ر' 'و' 'ی' 'ه' ' ' 'س' 'پ' 'ا' 'ه' ' ' '_EOM_']


In [5]:
word_index = dict((word, i) for i, word in enumerate(dictionary))
index_word = dict((i, word) for i, word in enumerate(dictionary))


In [12]:
print (word_index["_EOM_"])
print (index_word[word_index["_EOM_"]])

8
ر


In [21]:
x_train_index, y_train_index = [], []
for (x_sen, y_sen) in zip(x_train, y_train):
    x_train_index.append([word_index[elem] for elem in x_sen])
    y_train_index.append([word_index[elem] for elem in y_sen])

x_test_index, y_test_index = [], []
for (x_sen, y_sen) in zip(x_test, y_test):
    x_test_index.append([word_index[elem] for elem in x_sen])
    y_test_index.append([word_index[elem] for elem in y_sen])

In [41]:
print (x_train_index[0])
print (y_train_index[0])
print (len(x_train_index))
print (len(y_train_index))
print (len(x_test_index))
print (len(y_test_index))
print (len(x_train_index[0]))
print (len(y_train_index[0]))

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 22, 17, 2, 16, 33, 2, 16, 19, 36, 37, 18, 16, 33, 22, 16, 2, 22, 36, 27, 16, 28, 34, 37, 27, 16, 8]
[1, 16, 30, 17, 2, 33, 36, 16, 30, 35, 37, 7, 16, 37, 9, 16, 34, 36, 16, 18, 37, 19, 16, 22, 16, 23, 37, 8, 5, 5, 5, 5, 5, 5, 5, 5]
44648
44648
4961
4961
37
36


In [28]:
def make_batches(x, y, batch_size):
    n = len(y)
    steps = n // batch_size
    if n % batch_size != 0:
        steps += 1
    x_batches = np.array_split(x, steps)
    y_batches = np.array_split(y, steps)
    return x_batches, y_batches

In [36]:
lr = 1e-4
hidden_dim = 128


tf.reset_default_graph()

train_encoder_placeholder = tf.placeholder(shape = [None, None], dtype=tf.int32)
train_decoder_placeholder = tf.placeholder(shape = [None, None], dtype=tf.int32)

train_encoder = tf.one_hot(train_encoder_placeholder, depth = len(dictionary), 
                           on_value=1.0, off_value=0.0, axis=-1)
train_decoder = tf.one_hot(train_decoder_placeholder, depth = len(dictionary), 
                           on_value=1.0, off_value=0.0, axis=-1)

encoder_cell = tf.contrib.rnn.LSTMCell(hidden_dim)
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, train_encoder, dtype=tf.float32)

decoder_cell = tf.contrib.rnn.LSTMCell(hidden_dim, state_is_tuple = encoder_state)
decoder_outputs, decoder_state = tf.nn.dynamic_rnn(encoder_cell, train_decoder, dtype=tf.float32)

W = tf.get_variable(name = 'w',shape = (hidden_dim, len(dictionary)),initializer = tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name = 'b',shape = (len(dictionary),),initializer=tf.contrib.layers.xavier_initializer())
logits = tf.einsum('ijk,kl', decoder_outputs, W)+ b 

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=train_decoder))
trainer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cross_entropy)

In [39]:
########## Training ##########
batch_size =32  
x_train_batches, y_train_batches = make_batches(x_train_index, y_train_index, batch_size)
# print(y_train_batches[1])
epochs = 15
N_Batches = len(x_train_batches)

train_loss = np.zeros(epochs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(epochs):
    print()
    print("*" * 10, "Epoch %3i"%epoch, "*" * 10)

    batch_loss = np.zeros(N_Batches)

    for n in range(N_Batches):
        _, batch_loss[n] = sess.run([trainer, cross_entropy],
                                    {train_encoder_placeholder: x_train_batches[n], 
                                     train_decoder_placeholder: y_train_batches[n]})
        #print("Batch %3d/%d: Acc:%0.4f , Loss:%0.4f"%(n, N_Batches, batch_acc[n], batch_loss[n]))

    train_loss[epoch] = np.mean(batch_loss)
    print("Train:\tLoss= %0.4f"%(train_loss[epoch]))


********** Epoch   0 **********
Train:	Loss= 2.1007

********** Epoch   1 **********
Train:	Loss= 1.0056

********** Epoch   2 **********
Train:	Loss= 0.2283

********** Epoch   3 **********
Train:	Loss= 0.0555

********** Epoch   4 **********
Train:	Loss= 0.0195

********** Epoch   5 **********
Train:	Loss= 0.0080

********** Epoch   6 **********
Train:	Loss= 0.0036

********** Epoch   7 **********
Train:	Loss= 0.0017

********** Epoch   8 **********
Train:	Loss= 0.0009

********** Epoch   9 **********
Train:	Loss= 0.0005

********** Epoch  10 **********
Train:	Loss= 0.0003

********** Epoch  11 **********
Train:	Loss= 0.0002

********** Epoch  12 **********
Train:	Loss= 0.0001

********** Epoch  13 **********
Train:	Loss= 0.0001

********** Epoch  14 **********
Train:	Loss= 0.0000


In [66]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, batch_size):
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)

        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.placeholder(tf.int32, [None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])

        with tf.variable_scope("encoder_embeddings"):
            encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
            encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
            main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])

        with tf.variable_scope("decoder_embeddings"):
            decoder_input = batch_size
            decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
            decoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, decoder_input)

        with tf.variable_scope("encoder"):
            rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
            _, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded,
                                              dtype = tf.float32)

        with tf.variable_scope("decoder"):
            rnn_cells_dec = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
            outputs, _ = tf.nn.dynamic_rnn(rnn_cells_dec, decoder_embedded,
                                           initial_state = last_state,
                                           dtype = tf.float32)

        with tf.variable_scope("logits"):
            self.logits = tf.layers.dense(outputs,to_dict_size)
            print(self.logits)
            masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)

        with tf.variable_scope("cost"):
            self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.logits,
                                                         targets = self.Y,weights = masks)

        with tf.variable_scope("optimizer"):
            self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)

In [67]:
size_layer = 128
num_layers = 1
embedded_size = 25
learning_rate = 0.001
batch_size = 32
epoch = 50

In [68]:
def check_accuracy(logits, Y):
    acc = 0
    for i in range(logits.shape[0]):
        internal_acc = 0
    for k in range(len(Y[i])):
        if Y[i][k] == logits[i][k]:
            internal_acc += 1
        acc += (internal_acc / len(Y[i]))
    return acc / logits.shape[0]

In [69]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary),
                len(dictionary), learning_rate, batch_size)
sess.run(tf.global_variables_initializer())
seq_x = [len(x_train_index[0]) * batch_size]
seq_y = [len(y_train_index[0]) * batch_size]
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, (len(text_from) // batch_size) * batch_size, batch_size):
        batch_x = tf.one_hot(x_train_index[k: k+batch_size], depth = len(dictionary), 
                           on_value=1.0, off_value=0.0, axis=-1)
        batch_y = tf.one_hot(y_train_index[k: k+batch_size], depth = len(dictionary), 
                           on_value=1.0, off_value=0.0, axis=-1)
        predicted, loss, _ = sess.run([tf.argmax(model.logits,2), model.cost, model.optimizer],
                                      feed_dict={model.X:batch_x,
                                                 model.Y:batch_y,
                                                 model.X_seq_len:seq_x,
                                                 model.Y_seq_len:seq_y})
        total_loss += loss
        total_accuracy += check_accuracy(predicted,batch_y)
        
    total_loss /= (len(text_from) // batch_size)
    total_accuracy /= (len(text_from) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

/Users/alireza/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: Expected input tensor Tensor("decoder_embeddings/embedding_lookup/Identity:0", shape=(25,), dtype=float32) to have rank at least 2